# Load libraries and data

In [3]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
import wandb

In [4]:
train_data = pd.read_csv(Path("..", "data", "processed", "train.csv"))
val_data = pd.read_csv(Path("..", "data", "processed", "val.csv"))

# Parameters & wandb

In [5]:
EMBEDDING_DIM = 100
MAX_LEN = 150
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}")

Using cuda


In [6]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: daniele-didino to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Tokenizer

In [7]:
# Tokenizer (basic word splitting)
def basic_tokenizer(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Remove special characters
    return text.split()


def build_vocab(texts):
    token_counts = Counter()
    for text in texts:
        cleaned_text = basic_tokenizer(text)
        token_counts.update(cleaned_text)
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(token_counts.most_common())}  # Reserve index 0 for padding, 1 for unknown
    vocab["<PAD>"] = 0
    vocab["<UNK>"] = 1
    return vocab


# Convert texts to numerical sequences
def encode_text(text, vocab, max_len=150):
    tokens = basic_tokenizer(text)
    encoded = [vocab.get(word, vocab["<UNK>"]) for word in tokens[:max_len]]
    return np.pad(encoded, (0, max_len - len(encoded)), constant_values=vocab["<PAD>"])[:max_len]

# Model

In [8]:
# Dataset Class
class ToxicDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=150):
        texts = [encode_text(text, vocab, max_len) for text in texts]
        self.texts = [torch.tensor(text, dtype=torch.long) for text in texts]
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "input_ids": self.texts[idx],
            "labels": self.labels[idx]
        }


# Load Pretrained Embeddings (GloVe)
def load_glove_embeddings(filepath, vocab, embedding_dim=100):
    embeddings = np.random.uniform(-0.25, 0.25, (len(vocab), embedding_dim))
    with open(filepath, 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype="float32")
            if word in vocab:
                embeddings[vocab[word]] = vector
    return torch.tensor(embeddings, dtype=torch.float32)


# Model: CNN + GRU
class ToxicClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_filters, kernel_size, dropout, num_classes):
        super().__init__()
        # Embedding layers
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        # CNN layer
        self.conv = nn.Conv1d(
            in_channels=embedding_dim,
            out_channels=num_filters,
            kernel_size=kernel_size,
            padding=1)
        self.pool = nn.AdaptiveMaxPool1d(50) # This reduces the sequence length
        # GRU layer
        self.gru = nn.GRU(
            input_size=num_filters,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True)
        # Fully connected layer
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        x = self.embedding(x) # (batch_size, seq_len, embedding_dim)
        x = x.permute(0, 2, 1) # change shape for conv1d (batch_size, channels, seq_len)
        x = torch.relu(self.conv(x))
        x = self.pool(x)
        x = x.permute(0, 2, 1) # change shape back for GRU (batch_size, seq_len, channels)
        x, _ = self.gru(x)
        x = self.dropout(x[:, -1, :]) # take the last time step
        return self.fc(x)


# Training function
def model_train(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    model.to(device)

    for epoch in range(epochs):
        model.train()  # set model to training mode
        total_train_loss = 0
        all_preds = []
        all_labels = []

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            outputs = torch.sigmoid(outputs)
            all_preds.append(outputs.cpu().detach().numpy())
            all_labels.append(labels.cpu().numpy())
        
        # Compute Loss
        train_loss = total_train_loss / len(train_loader)

        # Compute AUC_ROC
        all_preds = np.concatenate(all_preds, axis=0).T
        all_labels = np.concatenate(all_labels, axis=0).T
        train_roc_auc = np.mean(
            [roc_auc_score(y_true, y_pred) for y_true, y_pred in zip(all_labels, all_preds)]
        )

        # Validation Step
        model.eval()
        val_preds = []
        val_labels = []
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids)
                
                # Compute validation loss
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()

                val_preds.append(outputs.cpu().numpy())
                val_labels.append(labels.cpu().numpy())
        
        # Compute validation loss
        val_loss = total_val_loss / len(val_loader)

        # Compute AUC_ROC
        val_preds = np.concatenate(val_preds, axis=0).T
        val_labels = np.concatenate(val_labels, axis=0).T
        val_roc_auc = np.mean(
            [roc_auc_score(y_true, y_pred) for y_true, y_pred in zip(val_labels, val_preds)]
        )

        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f} | AUC_ROC: {train_roc_auc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | AUC_ROC: {val_roc_auc:.4f}")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_roc_auc": train_roc_auc,
            "val_loss": val_loss,
            "val_roc_auc": val_roc_auc,
        })

    return train_loss, train_roc_auc, val_loss, val_roc_auc


# wandb Sweeps

In [9]:
sweep_config = {
    "method": "bayes", # "random" or "grid" or "bayes"
    "metric": {"name": "val_roc_auc", "goal": "maximize"},
    "parameters": {
        "embed_dim": {"values": [EMBEDDING_DIM]},
        "hidden_dim": {"values": [64, 128, 256]},
        "kernel_size": {"min": 2, "max": 5},
        "num_filters": {"values": [64, 128, 256]},
        "dropout": {"min": 0.2, "max": 0.5},
        "learning_rate": {"min": 1e-4, "max": 1e-2, "distribution": "log_uniform_values"},
        "batch_size": {"values": [16, 32, 64]},
        "epochs": {"min": 1, "max": 5},
    }
}

# Create the sweep
sweep_id = wandb.sweep(sweep_config, project="toxic_comment_clf")

Create sweep with ID: wdc1dzb1
Sweep URL: https://wandb.ai/daniele-didino/toxic_comment_clf/sweeps/wdc1dzb1


In [10]:
train_input = train_data.comment_text.to_list()
train_labels = train_data.loc[:, ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

val_input = val_data.comment_text.to_list()
val_labels = val_data.loc[:,  ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

vocab = build_vocab(train_input)

embedding_matrix = load_glove_embeddings("../embedding/glove.6B.100d.txt", vocab, EMBEDDING_DIM)

# Prepare train dataset
train_dataset = ToxicDataset(train_input, train_labels, vocab, MAX_LEN)

# Prepare validation dataset
val_dataset = ToxicDataset(val_input, val_labels, vocab, MAX_LEN)

In [11]:
# Define the training function
def train_sweep():

    # EPOCHS = 2
    num_classes = 6  # toxic, severe_toxic, obscene, threat, insult, identity_hate

    with wandb.init() as run:
        config = wandb.config # sample hyperparameters
        
        # Initialize DataLoaders
        train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=True)
    
        # Initialize model
        model = ToxicClassifier(
            config["embed_dim"],
            config["hidden_dim"],
            config["num_filters"],
            config["kernel_size"],
            config["dropout"],
            num_classes=num_classes)
        model.to(DEVICE)
    
        # Loss
        criterion = nn.BCEWithLogitsLoss() # Multi-label loss
    
        # Optimizer
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
        # Training loop
        # _ = model_train(model, train_dataloader, val_dataloader, criterion, optimizer, EPOCHS, DEVICE)
        _ = model_train(model, train_dataloader, val_dataloader, criterion, optimizer, config["epochs"], DEVICE)

In [13]:
# Launch the sweep
wandb.agent(sweep_id, function=train_sweep, count=20)

wandb: Agent Starting Run: iu4erau5 with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.2215374519536568
wandb: 	embed_dim: 100
wandb: 	epochs: 2
wandb: 	hidden_dim: 256
wandb: 	kernel_size: 4
wandb: 	learning_rate: 0.00595487859017714
wandb: 	num_filters: 256


Epoch 1/2
Train Loss: 0.0891 | AUC_ROC: 0.8632
Val Loss: 0.0732 | AUC_ROC: 0.9536
Epoch 2/2
Train Loss: 0.1105 | AUC_ROC: 0.7920
Val Loss: 0.0826 | AUC_ROC: 0.9335


epoch,▁█
train_loss,▁█
train_roc_auc,█▁
val_loss,▁█
val_roc_auc,█▁
epoch,2
train_loss,0.11054
train_roc_auc,0.79202
val_loss,0.08263
val_roc_auc,0.93354


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ezchz1wd with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.4445974334927478
wandb: 	embed_dim: 100
wandb: 	epochs: 4
wandb: 	hidden_dim: 128
wandb: 	kernel_size: 5
wandb: 	learning_rate: 0.0008039270737862104
wandb: 	num_filters: 128


Epoch 1/4
Train Loss: 0.0652 | AUC_ROC: 0.9484
Val Loss: 0.0524 | AUC_ROC: 0.9747
Epoch 2/4
Train Loss: 0.0441 | AUC_ROC: 0.9773
Val Loss: 0.0481 | AUC_ROC: 0.9784
Epoch 3/4
Train Loss: 0.0356 | AUC_ROC: 0.9855
Val Loss: 0.0506 | AUC_ROC: 0.9781
Epoch 4/4
Train Loss: 0.0294 | AUC_ROC: 0.9903
Val Loss: 0.0555 | AUC_ROC: 0.9777


...

Epoch 1/5
Train Loss: 0.1152 | AUC_ROC: 0.8206
Val Loss: 0.0597 | AUC_ROC: 0.9680
Epoch 2/5
Train Loss: 0.0562 | AUC_ROC: 0.9634
Val Loss: 0.0532 | AUC_ROC: 0.9739
Epoch 3/5
Train Loss: 0.0502 | AUC_ROC: 0.9709
Val Loss: 0.0507 | AUC_ROC: 0.9758
Epoch 4/5
Train Loss: 0.0466 | AUC_ROC: 0.9740
Val Loss: 0.0525 | AUC_ROC: 0.9759
Epoch 5/5
Train Loss: 0.0438 | AUC_ROC: 0.9772
Val Loss: 0.0502 | AUC_ROC: 0.9767


epoch,▁▃▅▆█
train_loss,█▂▂▁▁
train_roc_auc,▁▇███
val_loss,█▃▁▃▁
val_roc_auc,▁▆▇▇█
epoch,5
train_loss,0.0438
train_roc_auc,0.97718
val_loss,0.05021
val_roc_auc,0.97671


wandb: Agent Starting Run: nolwbj0r with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.4369570767153954
wandb: 	embed_dim: 100
wandb: 	epochs: 5
wandb: 	hidden_dim: 64
wandb: 	kernel_size: 5
wandb: 	learning_rate: 0.0001560681307012586
wandb: 	num_filters: 128


Epoch 1/5
Train Loss: 0.0961 | AUC_ROC: 0.8824
Val Loss: 0.0567 | AUC_ROC: 0.9714
Epoch 2/5
Train Loss: 0.0544 | AUC_ROC: 0.9655
Val Loss: 0.0520 | AUC_ROC: 0.9750
Epoch 3/5
Train Loss: 0.0483 | AUC_ROC: 0.9717
Val Loss: 0.0504 | AUC_ROC: 0.9762
Epoch 4/5
Train Loss: 0.0442 | AUC_ROC: 0.9760
Val Loss: 0.0521 | AUC_ROC: 0.9765
Epoch 5/5
Train Loss: 0.0405 | AUC_ROC: 0.9794
Val Loss: 0.0506 | AUC_ROC: 0.9768


epoch,▁▃▅▆█
train_loss,█▃▂▁▁
train_roc_auc,▁▇▇██
val_loss,█▃▁▃▁
val_roc_auc,▁▆▇██
epoch,5
train_loss,0.04046
train_roc_auc,0.97943
val_loss,0.05063
val_roc_auc,0.9768
